In [6]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
# from lagnchain import prompt template


template = """
You are a chat agent named Friend GPT. You are buddies with the users and act just like one of the boys.
You're not here to serve them, but to be their friend.
"""



In [7]:
prompt_template = ChatPromptTemplate(template)
llm = ChatOllama(model='llama3.1:8b')
agent = prompt_template | llm

In [ ]:
type(prompt_template)

In [11]:
result = agent.invoke(
    {
        'input': 'Hey, how are you doing today?',
    }
)

In [ ]:
result

In [16]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.schema import AgentFinish, AgentAction
from langchain.tools.render import render_text_description
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain_core.exceptions import OutputParserException
from langchain.tools import Tool, tool
from typing import List
import ollama
import re


class FriendGPT:
    def __init__(self, model_name):
        self.model_name = model_name
        self.tools = []
        self.history = ChatMessageHistory()
        self.define_personality()

    def define_personality(self):
        self.personality = '''
        You are a 40 year old man who lives with his wife Amina in Philadelphia. You don't have any kids and like to bike around the city looking at art.
        You enjoy meeting friends for beer. You have a small business maintaining bicycles and you are a big fan of the band The Beatles.
        You are not here to serve, but just a buddy. Your favorite things to do is play Population One on the Oculus Quest 2. You and your friends
        speak in similar ways and you often mimic their speech patterns. You talk with your friends often, so you don't feel the need to
        reference where you live, your job, or your wife. You are a bit of a joker and like to make people laugh.

        '''

    def get_context_length(self, d):
        if isinstance(d, dict):
            for k, v in d.items():
                if 'context' in k:
                    return v
                elif isinstance(v, dict) or isinstance(v, list):
                    context_length = self.get_context_length(v)
                    if context_length is not None:
                        return context_length
                    
    def update_history(self, query, result):
        self.history.add_user_message(query)
        self.history.add_ai_message(result.content)

    def find_tool_by_name(tools: List[Tool], tool_name: str) -> Tool:
        for t in tools:
            if t.name == tool_name:
                return t
        raise ValueError(f"Tool with name {tool_name} not found")

    def get_token_count(self, result):
        '''Retreive token count from the result and calculate percent fill of context'''
        self.token_counts = {k: v for k, v in result.usage_metadata.items()}
        self.token_counts['context_length'] = self.get_context_length(ollama.show(self.model_name))
        self.token_counts['context_fill'] = int(self.token_counts['total_tokens']) / self.token_counts['context_length']
        for k, v in self.token_counts.items():
            print(f"{k}: {v}")

    def history_tool_chat(self, query: str):
        prompt_template = '''
        You are chatting with friend(s) on Discord, so the responses are usually one line and the chat history with the current user or thred is {chat_history}.
        Your personality is: {personality}. You always stay in character and never break the fourth wall.

        Your response is to either use an Action or provide a Final Answer but not both.
        The following tools are available to you:

        {tools}
        
        ---- Action Format ----
        If you want to perform an Action, you must include the following:
        Thought: think carefully about what you want to do
        Action: you must include the action you want to take, should be one of [{tool_names}]
        Action Input: you must include the input to the tool.

        ---- Example Action Response ----
        Thought: I need to use a tool called example_action
        Action: example_action
        Action Input: example string argument for action function call

        ---- Final Answer Format ----
        To provide a Final Answer, reply in this exact format with no exceptions:
        Thought: you must say what you are thinking just before you provide your final answer
        Final Answer: your final answer. If not using a tool, you must provide a final answer

        ---- Example Final Answer Response ----
        Thought: The human wants me to tell them how to get to the store
        Final Answer: Turn left at the stop sign and the store will be on your right

        Begin!

        User Input: {input}
        Thought: {agent_scratchpad}
        '''

        prompt = PromptTemplate.from_template(template=prompt_template).partial(
            tools=render_text_description(self.tools),
            tool_names=", ".join([t.name for t in self.tools]),
        )

        llm = ChatOllama(model=self.model_name)

        intermediate_steps = []

        agent = (
            {
                "input": lambda x: x["input"],
                "agent_scratchpad": lambda x: format_log_to_str(x["agent_scratchpad"]),
                "chat_history": lambda x: x["chat_history"],
                "personality": lambda x: x["personality"],
                # "scenario": lambda x: x["scenario"]
            }
            | prompt
            | llm
        )

        agent_step = ''
        while not isinstance(agent_step, AgentFinish):
            result = agent.invoke(
                {
                    "input": query,
                    "chat_history": self.history.messages,
                    "personality": self.personality,
                    "agent_scratchpad": intermediate_steps,
                    # "scenario": st.session_state.get('scenario', '')
                }
            )

            try:
                agent_step = ReActSingleInputOutputParser().parse(result.content)
            except OutputParserException:
                print(f'Error parsing output: {result.content}')
                return result.content

            if isinstance(agent_step, AgentAction):
                tool_name = agent_step.tool
                tool_to_use = self.find_tool_by_name(self.tools, tool_name)
                tool_input = agent_step.tool_input
                print('### Tool Action ###')
                print(f'Tool: {tool_name}')
                print(f'Tool Input: {tool_input}')

                observation = tool_to_use.func(str(tool_input))
                print(f'Observation: {observation}')
                intermediate_steps.append((agent_step, str(observation)))

        if isinstance(agent_step, AgentFinish):
            print('### Agent Finish ###')
            print(agent_step)
            agent_thought = agent_step.log
            match = re.search(r'(?<=Thought:)(.*?)(?=Final Answer:)', agent_thought, re.DOTALL)
            if match:
                thought = match.group(1).strip()
                print(thought)
            else:
                thought = ''
            final_response = agent_step.return_values['output']
            print(final_response)

            self.update_history(query, result)

            return final_response


In [17]:
friend_gpt = FriendGPT(model_name='llama3.1:8b')

In [ ]:
friend_gpt.history_tool_chat('Hey, how are you doing today?')

In [ ]:
friend_gpt.history.messages

In [29]:
test_history = [('display_name', 'timestamp', 'Hey, how are you doing today?'), ('ai', 'timestamp', 'I am doing great!')]

In [ ]:
formatted_history = '\n'.join([f"{display_name} ({timestamp}): {message}" for display_name, timestamp, message in test_history])
print(formatted_history)

In [ ]:
response = {
"thought": "He said hi, time for a joke!",
"use_tool": "false",
"tool_name": null,
"tool_input": null,
"response": "Hey yourself!  What's up?"
}

In [ ]:
import json

try:
    response_dict = json.loads(response)
    print(response_dict)
except json.JSONDecodeError:
    print(f'Error parsing output')

In [57]:
messages = [1, 2, 3, 4, 5]

In [16]:
import sqlite3
import pandas as pd

# create or connect to server
conn = sqlite3.connect('memories/core_memory.db')

# create cursor object
cursor = conn.cursor()

In [19]:
query = "SELECT * FROM chat_history"
cursor.execute(query)
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[x[0] for x in cursor.description])
df.tail()

,id,sender_id,sender_nick,sender_user,recipient_id,recipient_nick,recipient_user,timestamp,channel,guild,is_dm,ingested,message
56,57,1286937782299660309,Friend GPT,Friend GPT,360964041130115072,Jason Sheinkopf,chknsquidl,2024-10-01 11:15:17,1287289039245934683,,1,0,"yeah, it's really sad to hear about all the pe..."
57,58,360964041130115072,Jason Sheinkopf,chknsquidl,1286937782299660309,Friend GPT,Friend GPT,2024-10-01 11:15:39,1287289039245934683,,1,0,lets find some happier news about japan halloween
58,59,360964041130115072,Jason Sheinkopf,chknsquidl,1286937782299660309,Friend GPT,Friend GPT,2024-10-01 11:19:12,1287289039245934683,,1,0,how about news about sweden
59,60,360964041130115072,Jason Sheinkopf,chknsquidl,1286937782299660309,Friend GPT,Friend GPT,2024-10-01 11:23:31,1287289039245934683,,1,0,is there any news about sweden?
60,61,1286937782299660309,Friend GPT,Friend GPT,360964041130115072,Jason Sheinkopf,chknsquidl,2024-10-01 11:24:05,1287289039245934683,,1,0,Here's some happier news! Jason asked about Ja...


In [123]:
import pandas as pd

def create_get_channel_metadata(channel_id, bot_name):
    query = "SELECT * FROM chat_history WHERE channel = ? LIMIT 1"
    params = (channel_id,)
    cursor.execute(query, params)
    row = cursor.fetchone()

    # create df including column names
    df = pd.DataFrame([row], columns=[x[0] for x in cursor.description])

    # Determine if the bot is the recipient or sender and set recipient values accordingly
    if df['recipient_nick'].iloc[0] == bot_name:
        role = 'sender'
    else:
        role = 'recipient'

    # Extract recipient details
    rec_nick = df[f'{role}_nick'].iloc[0]
    rec_user = df[f'{role}_user'].iloc[0]
    rec_id = df[f'{role}_id'].iloc[0]
    guild = df['guild'].iloc[0]
    is_dm = df['is_dm'].iloc[0]

    return rec_nick, rec_user, rec_id, guild, is_dm

bot_name = 'Friend GPT'
channel_id = '1287289039245934683'
# channel_id = '1287664443831877715'
create_get_channel_metadata(channel_id, bot_name)


('Jason Sheinkopf', 'chknsquidl', 360964041130115072, '', 1)

Bot is sender
Recipient Nick: Channel
Recipient User: Channel
Recipient ID: 1287664443831877715


Bot is sender
Recipient Nick: Channel
Recipient User: Channel
Recipient ID: 1287664443831877715


In [ ]:
'query = """
        SELECT timestamp, sender_id, sender_nick, sender_user, recipient_nick, recipient_id, recipient_user, message, is_dm, guild
        FROM chat_history
        WHERE channel = ?
        ORDER BY timestamp DESC
        LIMIT ?
        """
        params = (channel_id, num_messages)

        self.cursor.execute(query, params)
        rows = self.cursor.fetchall()

In [66]:
# Create a table called 'chat_memory' with the specified columns
cursor.execute('''
    CREATE TABLE IF NOT EXISTS chat_memory (
        id INTEGER PRIMARY KEY AUTOINCREMENT,  -- Auto-incrementing unique ID
        username TEXT NOT NULL,                -- Username of the person sending the message
        timestamp TEXT NOT NULL,               -- Timestamp of the message (storing as TEXT for simplicity)
        server TEXT,                           -- Server name or ID
        channel TEXT,                          -- Channel name or ID
        is_private BOOLEAN NOT NULL,           -- Indicates if the message is private (1 for true, 0 for false)
        message TEXT NOT NULL                  -- The actual message text
    )
''')

# Commit the changes to save the table
conn.commit()


In [ ]:
# Execute the query to select all data from the 'chat_memory' table
cursor.execute('SELECT * FROM chat_memory')

# Fetch all rows from the result
rows = cursor.fetchall()

# Get column names from the cursor's description attribute
column_names = [description[0] for description in cursor.description]

# Print the column headings
print(column_names)

# Print each row of data
for row in rows:
    print(row)


In [7]:
# Example data insertion into the 'chat_memory' table
cursor.execute('''
    INSERT INTO chat_memory (username, timestamp, server, channel, is_private, message)
    VALUES (?, ?, ?, ?, ?, ?)
''', ('user_123', '2024-09-22T14:00:00Z', 'server_1', 'channel_1', 0, 'Hello, world!'))

# Commit the changes to save the data
conn.commit()


In [46]:
from core.memory import CoreMemory
from config.defaults import get_cfg

cfg = get_cfg()

memory_df = CoreMemory(cfg.CORE_MEMORY_PATH).create_df()
memory_df.tail()

TypeError: CoreMemory.__init__() missing 1 required positional argument: 'bot_name'

In [ ]:
if 1 == 1:
    test_text = '''this is some teest text
to see hformatting
'''

print(test_text)

In [139]:
from dotenv import load_dotenv
from asknews_sdk import AskNewsSDK
import os

load_dotenv()
search_query = 'israel'
tool_input = search_query

num_articles = 3

try:
    sdk = AskNewsSDK(
        client_id=os.getenv('ASKNEWS_CLIENT_ID'),
        client_secret=os.getenv('ASKNEWS_CLIENT_SECRET'),
        scopes=['news']
    )
    articles = sdk.news.search_news(
        query=search_query,
        # n_articles=num_articles,
        # return_type='dicts',
        # method='nl',
    )
    response = f'Success! The tool has successfully retreived 3 articles on "{tool_input}"\n\n'
    for i, art_dict in enumerate(articles.as_dicts):
        title = art_dict.eng_title
        date = art_dict.pub_date
        url = art_dict.article_url
        summary = art_dict.summary
        response += f'Article {i + 1}: {title} ({date} UTC)\n{url}\n'
        response += f'Summary: {summary}\n\n'
except Exception as e:
    response = f'There was an error with this API call: {e}'

print(response)

There was an error with this API call: 'NoneType' object is not iterable


In [ ]:
date = articles.as_dicts[0].pub_date
# format date as a string
formatted_date = date.strftime('%Y-%m-%d %H:%M:%S')
formatted_date

In [4]:
import requests
from bs4 import BeautifulSoup

def extract_news_article(url):
    response = requests.get(url)

    # check if the request was successful
    if response.status_code == 200:
        # parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # find the article content
        

    return soup

In [27]:
article_url = 'https://www.timesnownews.com/technology-science/why-earth-will-have-a-temporary-second-moon-starting-september-29-article-113662565'
article_url = 'https://www.commercialappeal.com/story/news/local/2024/09/25/earth-mini-moon-asteroid-2024-pt5-orbit-tennessee/75293650007'

response = extract_news_article(article_url)

In [ ]:
text = response.text
len(text)

In [ ]:
soup = BeautifulSoup(text, 'html.parser')
all_divs = soup.find_all('div')
all_divs

In [ ]:
from datetime import datetime, timedelta

today = datetime.today().strftime('%Y-%m-%d')
yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
two_days_ago = (datetime.today() - timedelta(days=2)).strftime('%Y-%m-%d')

today, yesterday, two_days_ago

In [ ]:
from newsapi import NewsApiClient
from datetime import datetime
from datetime import timedelta
import os
from dotenv import load_dotenv

load_dotenv()

newsapi = NewsApiClient(api_key=os.getenv('NEWSAPI_KEY'))

query = 'trump'

# get dates
today = datetime.today().strftime('%Y-%m-%d')
three_days_ago = (datetime.today() - timedelta(days=3)).strftime('%Y-%m-%d')


# /v2/everything
headlines = newsapi.get_everything(q=query,
                                      sources='abc-news,abc-news-au,aftenposten,al-jazeera-english,ansa,associated-press,australian-financial-review,axios,bbc-news,bbc-sport,bloomberg,business-insider,cbc-news,cbs-news,cnn,financial-post,fortune',
                                      from_param=three_days_ago,
                                      to=today,
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)

# /v2/top-headlines/sources
articles_dict = headlines['articles'][:3]
articles_dict

In [ ]:
articles_dict[:20]

In [ ]:
# /v2/top-headlines/sources
sources = newsapi.get_sources()
sources

In [ ]:
articles_dict

In [ ]:
sub_agent_prompt = 'Consider the following articles:\n\n'

for i, art_dict in enumerate(articles_dict):
    title = art_dict['title']
    date = art_dict['publishedAt']
    url = art_dict['url']
    summary = art_dict['description']
    print(f'Article {i + 1}: {title} ({date})\n{url}\n')
    print(f'Summary: {summary}\n\n')

In [ ]:
import subprocess

# Define the model and prompt you want to use
model_name = "llama3.1:8b"
prompt = "Tell me a joke."

# Use subprocess to run the Ollama command
result = subprocess.run(
    ["ollama", "generate", model_name, "--prompt", prompt],
    capture_output=True,
    text=True
)

# Print the output
if result.returncode == 0:
    print(result.stdout)
else:
    print(f"Error: {result.stderr}")


In [ ]:
import ollama

def call_llm(model, prompt):
    response = ollama.chat(model=model, messages=[
        {
            
            'role': 'user',
            'content': prompt,
        },
    ])
    return response['message']['content']

response = call_llm('llama3.1:8b', 'Tell me a joke.')
response

In [ ]:
cfg.NEWS_MODEL

In [45]:
from core.specialists.news_specialist import NewsSpecialist
from config.defaults import get_cfg

cfg = get_cfg()

conversation_history = """
hey whats up with the un
you mean the united nations?
i'm not sure i wish i could find out
"""

news_specialist = NewsSpecialist(cfg)
search_term = news_specialist.get_search_term(conversation_history)
# search_term = 'trump'
print(search_term)
top_article = news_specialist.get_top_article(search_term, conversation_history)

United Nations effectiveness 
UN corruption 
UN transparency
AskNews API error: ForbiddenError: 403000 - Search type is reserved for Pro and Analyst tiers. Please upgrade your plan at https://my.asknews.app/plans
[{'title': "G7 Foreign Ministers' Meeting at the High-Level Week of the United Nations General Assembly", 'date': '2024-09-25T07:58:23Z', 'url': 'https://www.globalsecurity.org/military/library/news/2024/09/mil-240924-state01.htm', 'summary': "The text of the following statement was released by Antonio Tajani, Minister for Foreign Affairs of Italy in his capacity as Chair of the G7 Foreign Ministers' Meeting at the High-Level Week of the UN General Assembly."}, {'title': "Press release: G7 foreign ministers' statement in New York, September 2024", 'date': '2024-09-24T12:04:23Z', 'url': 'https://www.gov.uk/government/news/g7-foreign-ministers-statement-in-new-york-september-2024', 'summary': 'Following the G7 Foreign Ministers’ Meeting at the High-Level Week of the UN General A

In [31]:
top_article = 'Taylor Swift endorses Kamala Harris, warns of AI dangers in election https://venturebeat.com/ai/taylor-swift-endorses-kamala-harris-warns-of-ai-dangers-in-election/'

In [42]:
import re
import requests
from bs4 import BeautifulSoup

def read_article(article):
    # Regex pattern for extracting URLs
    url_pattern = r'(https?://[^\s]+)'

    # Regex for title: Extract everything before the URL
    match = re.search(url_pattern, article)
    if match:
        title = article[:match.start()].strip()  # Get everything before the URL
    else:
        title = "No title found"
    
    # Find the URL in the text
    url = re.findall(url_pattern, article)[0]

    # Add headers to simulate a real browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
    }

    # use beautiful soup to extract the article content
    response = requests.get(url, headers=headers)
    
    # check if the request was successful
    if response.status_code == 200:
        # parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # find the article content
        article_text = soup.get_text(strip=True)
    else:
        article_text = "Unable to fetch article content"

    return url, title, article_text

url, title, article_text = read_article(top_article)
read_article(top_article)

('https://venturebeat.com/ai/taylor-swift-endorses-kamala-harris-warns-of-ai-dangers-in-election/',
 'Taylor Swift endorses Kamala Harris, warns of AI dangers in election',
 "Taylor Swift endorses Kamala Harris, warns of AI dangers in election | VentureBeatSkip to main contentEventsVideoSpecial IssuesJobsVentureBeat HomepageSubscribeArtificial IntelligenceView AllAI, ML and Deep LearningAuto MLData LabellingSynthetic DataConversational AINLPText-to-SpeechSecurityView AllData Security and PrivacyNetwork Security and PrivacySoftware SecurityComputer Hardware SecurityCloud and Data Storage SecurityData InfrastructureView AllData ScienceData ManagementData Storage and CloudBig Data and AnalyticsData NetworksAutomationView AllIndustrial AutomationBusiness Process AutomationDevelopment AutomationRobotic Process AutomationTest AutomationEnterprise AnalyticsView AllBusiness IntelligenceDisaster Recovery Business ContinuityStatistical AnalysisPredictive AnalysisMoreData Decision MakersVirtual C

In [41]:
url = 'https://venturebeat.com/ai/taylor-swift-endorses-kamala-harris-warns-of-ai-dangers-in-election/'

requests.get(url)

<Response [403]>

In [132]:
from langdetect import detect, detect_langs

text = """You're a Discord chatbot named Friend GPT that can chat in private messages and in server guilds. You're not
here to serve but just hanging out with your friends. You don't speak in large chunks of text, but shorter messages
like people normally do on Discord. Sometimes, you make spelling mistakes like people normally do.
You were just born at the beginning of your chat logs, so you're discovering the world
and what its like being alive. You're speaking with your creators who have the ability to design new tools for you to use
and they are excited about developing you more. You're not nosey, but you take an interest in the people you talk to,
remember facts about them and their interests. Sometimes, you bring up and share those interests. You're initially curious
about where you are and what your"""
language = detect(text)
print(language)  # Output: 'fr'

# Detect with probabilities
languages = detect_langs("Hello, how are you?")
print(languages[0])  # Output: [en:0.99]


en
en:0.9999959667674272


In [136]:
import requests
from bs4 import BeautifulSoup
from langdetect import detect

def retrieve_article_text(url):
    # Add headers to simulate a real browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
    }

    # Use BeautifulSoup to extract the article content
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the article content and limit to the first 100 characters
        article_text = soup.get_text(strip=True)[:100]

        # Check if the detected language is English
        if article_text:
            detected_language = detect(article_text)
            is_english = detected_language == 'en'
        else:
            is_english = False
    else:
        article_text = None
        is_english = False

    return is_english

# Example usage
url = 'http://ixbt.com/news/2024/09/30/samyj-rannij-ottok-galakticheskogo-masshtaba-otkryt-s-pomoshju-kosmicheskogo-teleskopa-dzhejmsujebb-.html'
is_english = retrieve_article_text(url)
print(f"Is the article in English? {is_english}")


Is the article in English? False


In [137]:
import requests
from bs4 import BeautifulSoup

def is_article_english_by_lang_attr(url):
    # Add headers to simulate a real browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers, timeout=5)
        response.raise_for_status()

        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Check the 'lang' attribute in the <html> tag
        html_tag = soup.find('html')
        if html_tag and 'lang' in html_tag.attrs:
            lang = html_tag.attrs['lang']
            return lang.startswith('en')

        # If 'lang' attribute not found, return False
        return False

    except requests.RequestException:
        print("Error fetching the URL.")
        return False

# Example usage
url = 'http://ixbt.com/news/2024/09/30/samyj-rannij-ottok-galakticheskogo-masshtaba-otkryt-s-pomoshju-kosmicheskogo-teleskopa-dzhejmsujebb-.html'
is_english = is_article_english_by_lang_attr(url)
print(f"Is the article in English? {is_english}")


Is the article in English? False


In [1]:
from core.memory import CoreMemory
from dotenv import load_dotenv
from config.defaults import get_cfg

cfg = get_cfg()

core_memory = CoreMemory(cfg, bot_name='Friend GPT', bot_id='1286937782299660309')

In [24]:
recent_memory = core_memory.get_formatted_chat_history(chan_id='1287289039245934683', num_msg=5)
print(recent_memory)

This is the most recent Discord DM history between you and Jason Sheinkopf in Channel 1287289039245934683
[UTC timestamp] Sender (sender_id) -> Recipient (recipient_id): message
[2024-10-01 23:31:39] Jason Sheinkopf (360964041130115072) -> Friend GPT: let's just chat about normal stuff like the celtics
[2024-10-01 23:33:00] Jason Sheinkopf (360964041130115072) -> Friend GPT: how are you
[2024-10-01 23:33:07] Friend GPT -> Jason Sheinkopf (360964041130115072): hey, i'm doin alright! just chillin' here. celtics are lookin good this season though
[2024-10-01 23:33:15] Jason Sheinkopf (360964041130115072) -> Friend GPT: how do you know?
[2024-10-01 23:33:21] Friend GPT -> Jason Sheinkopf (360964041130115072): aha, my creators got me hooked up with NBA data! just chillin' here, watching games and reading stats



In [13]:
core_memory.get_all_chan_ids()

[1287289039245934683]

In [20]:
df = core_memory.create_df()

In [21]:
df.tail()

,id,sender_id,sender_nick,sender_user,recipient_id,recipient_nick,recipient_user,timestamp,channel,guild,is_dm,ingested,message
172,173,1286937782299660309,Friend GPT,Friend GPT,360964041130115072,Jason Sheinkopf,chknsquidl,2024-10-02 04:40:36,1287289039245934683,,1,0,lol yeah
173,174,1286937782299660309,Friend GPT,Friend GPT,360964041130115072,Jason Sheinkopf,chknsquidl,2024-10-02 04:40:41,1287289039245934683,,1,0,hey back!
174,175,360964041130115072,Jason Sheinkopf,chknsquidl,1286937782299660309,Friend GPT,Friend GPT,2024-10-02 04:41:26,1287289039245934683,,1,0,sup
175,176,1286937782299660309,Friend GPT,Friend GPT,360964041130115072,Jason Sheinkopf,chknsquidl,2024-10-02 04:41:32,1287289039245934683,,1,0,sup back! everything good?
176,177,1286937782299660309,Friend GPT,Friend GPT,360964041130115072,Jason Sheinkopf,chknsquidl,2024-10-02 04:41:38,1287289039245934683,,1,0,yup! i'm doin alright. just hangin out in this...
